In [1]:
import pandas as pd
import numpy as np
import pickle
import pandas_profiling
import scipy.sparse as sp
from sklearn.preprocessing import LabelEncoder
from keras import backend as K
from keras.models import Model, load_model
from keras import initializers
from keras.regularizers import l2
from keras.optimizers import Adam, SGD
from keras.layers import Embedding, Input, merge, Flatten, concatenate, multiply, Dense
from keras.utils import plot_model, CustomObjectScope
from keras.callbacks import ModelCheckpoint
# from tensorflow.keras.models import load_model
from utils import *
#
import os
os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/bin/'

Using TensorFlow backend.


In [2]:
df = get_df_from_mysql(database_name="StockRecommend", table_name="ncf_data")
print(df.head())
df.info()

                       main_account ShareCode  ShareBalance
0  0001898ab3584ff6a99ec11ffcb1f09c       CTG           1.0
1  0001898ab3584ff6a99ec11ffcb1f09c       FLC         401.0
2  0001898ab3584ff6a99ec11ffcb1f09c       PGC           5.0
3  0001898ab3584ff6a99ec11ffcb1f09c       PVI          50.0
4  0001898ab3584ff6a99ec11ffcb1f09c       PVS        2140.0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42660 entries, 0 to 42659
Data columns (total 3 columns):
main_account    42660 non-null object
ShareCode       42660 non-null object
ShareBalance    42660 non-null float64
dtypes: float64(1), object(2)
memory usage: 1000.0+ KB


In [3]:
# df.profile_report().to_file(output_file="data/stock.html")
# df.profile_report()

In [4]:
# Unique users, stocks
list_share = df.ShareCode.unique()
list_user = df.main_account.unique()
print(len(list_share))
len(list_user)

1191


13564

In [5]:
# Encoder accountCode
account_encoder = LabelEncoder()
df["account_encoder"] = account_encoder.fit_transform(df["main_account"])
account_dict = dict(zip(account_encoder.classes_, range(len(account_encoder.classes_))))
save_dict_to_pickle('data/account_dict.pickle', account_dict)
print(account_dict)
df.head()

{'0001898ab3584ff6a99ec11ffcb1f09c': 0, '00029e5f2da54460a1d7dd1a491d991f': 1, '00057207ba05405a911387bbcb3861bc': 2, '000a64c8a9d5465eba885a561e1b0ca9': 3, '001d09340ebb4b95af55684d8635a819': 4, '0023387d6b544f53a9306495d5576cae': 5, '00237505b9d5473186b894283286e3a7': 6, '0029fdde7ec841dbbf3c5166b50ed76d': 7, '002c1a2e3c054378b7261e680f19b5c4': 8, '002c9f8d8f9842b9959f3f52bcc3b5be': 9, '002d22cd64e2400387a989e266173f6d': 10, '00333607002b4ba3be5c9e3913597800': 11, '003e44f75bfb42309b547c4bcde24c84': 12, '003ecd779fd64432af51cb4a29e1646d': 13, '00406c885e5c4a3e89aa3c80276c636f': 14, '0047dc3eae7641728d03ada5566f979e': 15, '0057ef5491f24d83a5dd36fe819966ee': 16, '005a1809e7b745f383ce341bfcf7b30a': 17, '0060c535ccef4d44ab4771157d16e8ae': 18, '0062ac62fd0a485b93e8240ebf1b9107': 19, '00649b878d5a48cea1c86700384a181c': 20, '0065816b10b04f3d97e622b0a0b588a1': 21, '00659ff528be4d8d97fd511e077b4edc': 22, '00711725c8d44e308a9a2faf4690fdbc': 23, '00761c70a9f04eac8d4d272fa50a92bc': 24, '00781965

,main_account,ShareCode,ShareBalance,account_encoder
0,0001898ab3584ff6a99ec11ffcb1f09c,CTG,1.0,0
1,0001898ab3584ff6a99ec11ffcb1f09c,FLC,401.0,0
2,0001898ab3584ff6a99ec11ffcb1f09c,PGC,5.0,0
3,0001898ab3584ff6a99ec11ffcb1f09c,PVI,50.0,0
4,0001898ab3584ff6a99ec11ffcb1f09c,PVS,2140.0,0


In [6]:
# Encode shareCode
shareCode_encoder = LabelEncoder()
df["share_encoder"] = shareCode_encoder.fit_transform(df["ShareCode"])
share_dict = dict(zip(shareCode_encoder.classes_, range(len(shareCode_encoder.classes_))))
save_dict_to_pickle('data/share_dict.pickle', share_dict)
print(share_dict)
df.head()

{'AAA': 0, 'AAM': 1, 'AAT': 2, 'AAV': 3, 'ABC': 4, 'ABI': 5, 'ABR': 6, 'ABT': 7, 'ACB': 8, 'ACC': 9, 'ACL': 10, 'ACM': 11, 'ACV': 12, 'ADC': 13, 'ADP': 14, 'ADS': 15, 'AFX': 16, 'AGC': 17, 'AGF': 18, 'AGM': 19, 'AGP': 20, 'AGR': 21, 'AGX': 22, 'ALP': 23, 'ALT': 24, 'ALV': 25, 'AMD': 26, 'AME': 27, 'AMS': 28, 'AMV': 29, 'ANV': 30, 'APC': 31, 'APG': 32, 'API': 33, 'APP': 34, 'APS': 35, 'ARM': 36, 'ART': 37, 'ASA': 38, 'ASM': 39, 'ASP': 40, 'AST': 41, 'ATA': 42, 'ATB': 43, 'ATG': 44, 'AVC': 45, 'AVF': 46, 'AVS': 47, 'B82': 48, 'BAB': 49, 'BAL': 50, 'BAM': 51, 'BAS': 52, 'BAX': 53, 'BBC': 54, 'BBS': 55, 'BBT': 56, 'BCC': 57, 'BCE': 58, 'BCG': 59, 'BCM': 60, 'BCP': 61, 'BDG': 62, 'BDT': 63, 'BED': 64, 'BEL': 65, 'BFC': 66, 'BGM': 67, 'BGW': 68, 'BHA': 69, 'BHC': 70, 'BHN': 71, 'BHV': 72, 'BIC': 73, 'BID': 74, 'BII': 75, 'BKC': 76, 'BLF': 77, 'BLI': 78, 'BMC': 79, 'BMF': 80, 'BMI': 81, 'BMJ': 82, 'BMP': 83, 'BMS': 84, 'BOT': 85, 'BPC': 86, 'BPW': 87, 'BQB': 88, 'BRC': 89, 'BSG': 90, 'BSH': 9

,main_account,ShareCode,ShareBalance,account_encoder,share_encoder
0,0001898ab3584ff6a99ec11ffcb1f09c,CTG,1.0,0,201
1,0001898ab3584ff6a99ec11ffcb1f09c,FLC,401.0,0,320
2,0001898ab3584ff6a99ec11ffcb1f09c,PGC,5.0,0,675
3,0001898ab3584ff6a99ec11ffcb1f09c,PVI,50.0,0,745
4,0001898ab3584ff6a99ec11ffcb1f09c,PVS,2140.0,0,750


In [7]:
interaction_matrix = sp.dok_matrix((len(list_user) + 1, len(list_share) + 1), dtype=np.float32)
print(interaction_matrix.shape)
account_share_zip = list(zip(df['account_encoder'], df['share_encoder']))
for i in range(len(account_share_zip)):
    account = account_share_zip[i][0]
    share = account_share_zip[i][1]
    interaction_matrix[account, share] = 1
interaction_matrix.todense() # matrix if user invest stock = 1

(13565, 1192)


matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [8]:
# Hyper parameters
num_negatives = 4
num_epochs = 100
batch_size = 128
mf_dim = 32
layers = [64, 32, 16, 8]
reg_mf = 0
reg_layers = [0, 0, 0, 0]
learning_rate = 0.001
learner = Adam
verbose = 0
topK = 10
num_layer = len(layers)
evaluation_threads = 1
model_out_file = 'Pretrain/%s_NeuMF_%d_%s.h5' %('NCF', mf_dim, layers)

In [9]:
num_users, num_items = interaction_matrix.shape

In [10]:
def get_train_instances(interaction_matrix, num_negatives):
    user_input, item_input, labels = [],[],[]
    num_users = interaction_matrix.shape[0]
    for (u, i) in interaction_matrix.keys():
        # positive instance
        user_input.append(u)
        item_input.append(i)
        labels.append(1)
        # negative instances
        for t in range(num_negatives):
            j = np.random.randint(num_items)
            while (u, j) in interaction_matrix:
                j = np.random.randint(num_items)
            user_input.append(u)
            item_input.append(j)
            labels.append(0)
    return user_input, item_input, labels
user_input_data, item_input_data, labels_data = get_train_instances(interaction_matrix, num_negatives)
print(len(user_input_data))
print(len(item_input_data))
print(len(labels_data))

213300
213300
213300


In [11]:
# Get model
user_input = Input(shape=(1,), dtype='int64', name = 'user_input')
item_input = Input(shape=(1,), dtype='int64', name = 'item_input')
MF_Embedding_User = Embedding(input_dim = num_users, output_dim = mf_dim, name = 'mf_embedding_user',
                              embeddings_initializer = "RandomNormal", embeddings_regularizer = l2(reg_mf), input_length=1)
MF_Embedding_Item = Embedding(input_dim = num_items, output_dim = mf_dim, name = 'mf_embedding_item',
                              embeddings_initializer = "RandomNormal", embeddings_regularizer = l2(reg_mf), input_length=1) 
MLP_Embedding_User = Embedding(input_dim = num_users, output_dim = int(layers[0]/2), name = "mlp_embedding_user",
                              embeddings_initializer = "RandomNormal", embeddings_regularizer = l2(reg_layers[0]), input_length=1)
MLP_Embedding_Item = Embedding(input_dim = num_items, output_dim = int(layers[0]/2), name = 'mlp_embedding_item',
                              embeddings_initializer = "RandomNormal", embeddings_regularizer = l2(reg_layers[0]), input_length=1) 
mf_user_latent = Flatten()(MF_Embedding_User(user_input))
mf_item_latent = Flatten()(MF_Embedding_Item(item_input))
#
mlp_user_latent = Flatten()(MLP_Embedding_User(user_input))
mlp_item_latent = Flatten()(MLP_Embedding_Item(item_input))
mlp_vector = concatenate([mlp_user_latent, mlp_item_latent])
mf_vector = multiply([mf_user_latent, mf_item_latent]) # element-wise multiply
for idx in range(1, num_layer):
    layer = Dense(layers[idx], activity_regularizer= l2(reg_layers[idx]), activation='relu', name="layer%d" %idx)
    mlp_vector = layer(mlp_vector)
predict_vector = concatenate([mf_vector, mlp_vector])
prediction = Dense(1, activation='sigmoid', kernel_initializer='lecun_uniform', name = "prediction")(predict_vector)
model = Model(inputs=[user_input, item_input], outputs= prediction)
model.compile(optimizer=Adam(lr=learning_rate), loss='binary_crossentropy', metrics= ['accuracy'])
plot_model(model, to_file='model_architecture.png')
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
user_input (InputLayer)         (None, 1)            0                                            
__________________________________________________________________________________________________
item_input (InputLayer)         (None, 1)            0                                            
__________________________________________________________________________________________________
mlp_embedding_user (Embedding)  (None, 1, 32)        434080      user_input[0][0]                 
__________________________________________________________________________________________________
mlp_embedding_item (Embedding)  (None, 1, 32)        38144       item_input[0][0]                 
____________________________________________________________________________________________

In [12]:
best_hr = 0
checkpoint = ModelCheckpoint(filepath= './model/model-{loss:.4f}.hdf5', 
                            verbose=0, 
                            save_best_only= True, 
                            #save_weights_only= True,
                            monitor= 'loss', 
                            mode = 'min')
for epoch in range(num_epochs):
    # Fit model
    user_input_data, item_input_data, labels_data = get_train_instances(interaction_matrix, num_negatives)
    hist = model.fit([np.array(user_input_data), np.array(item_input_data)], #input
                    np.array(labels_data), # labels 
                    batch_size=batch_size, epochs=1, 
                    callbacks= [checkpoint], shuffle=True)

c:\users\natuh\appdata\local\programs\python\python36\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/1
213300/213300 [==============================] - 26s 123us/step - loss: 0.3458 - accuracy: 0.8559
Epoch 1/1
213300/213300 [==============================] - 23s 106us/step - loss: 0.3002 - accuracy: 0.8702
Epoch 1/1
213300/213300 [==============================] - 23s 108us/step - loss: 0.2415 - accuracy: 0.8987
Epoch 1/1
  6912/213300 [..............................] - ETA: 20s - loss: 0.1812 - accuracy: 0.9291

KeyboardInterrupt: 

In [ ]:
model = load_model("model/model-0.0092.hdf5")

In [ ]:
user = np.array([1372])
item = np.array([600])
print(user)
print(item)
model.predict([user, item])

In [ ]:
# # Too slow => api with 1 customer -> sorted list of prediction
# predicted_matrix = interaction_matrix
# for i in df['account_encoder']:
#     for j in df['share_encoder']:
#         if (i, j) not in predicted_matrix:
#             user = np.array([i])
#             item = np.array([j])
#             predicted_value = model.predict([user, item]) # [[3.886186e-10]]
#             predicted_matrix[i, j] = np.asscalar(predicted_value)
#             print(predicted_value)